In [1]:
import os # для работы с файлами, папками и процессами на уровне ос
from pathlib import Path
import cv2
from ultralytics import YOLO  # Для YOLOv8
import torch  # Для других моделей
import pandas as pd
import time
import numpy as np

In [2]:
# Пути
VIDEO_DIR = Path("./additional/video_detection")
RESULT_DIR = Path("./results")
RESULT_DIR.mkdir(exist_ok=True)

In [3]:
# Список видео (для оценки качества модели взяла три видосика, чтобы максимально разные, динамичные)
video_files = ["vid1.mp4", "vid2.mp4", "vid3.mp4"]

In [6]:
# Бойцы для сравнения (YOLOv5, YOLOv8, SSD, Faster R-CNN, Retina Net, Mask R-CNN)
# Все модели используют предобученные веса на COCO (80 классов, включая класс "person")

from ultralytics import YOLO
import torch
from torchvision.models.detection import (
    ssd300_vgg16,
    fasterrcnn_resnet50_fpn,
    retinanet_resnet50_fpn,
    maskrcnn_resnet50_fpn
)

MODELS = {
    # YOLOv5 (через torch.hub)
    "yolov5": torch.hub.load(
        'ultralytics/yolov5',  # репозиторий
        'yolov5s',             # маленькая версия (быстрая, подходит для real-time)
        pretrained=True        # автоматически подгружаем веса, обученные на COCO
    ).to(DEVICE).eval(),

    # YOLOv8 (через ultralytics)
    "yolov8": YOLO("yolov8n.pt"),  # nano-версия, оптимальна для real-time

    # SSD
    "ssd": ssd300_vgg16(pretrained=True).to(DEVICE).eval(),

    # Faster R-CNN
    "fasterrcnn": fasterrcnn_resnet50_fpn(pretrained=True).to(DEVICE).eval(),

    # RetinaNet
    "retinanet": retinanet_resnet50_fpn(pretrained=True).to(DEVICE).eval(),

    # Mask R-CNN
    "maskrcnn": maskrcnn_resnet50_fpn(pretrained=True).to(DEVICE).eval(),
}

Using cache found in C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-11-9 Python-3.10.19 torch-2.5.1 CUDA:0 (NVIDIA GeForce MX150, 2048MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [4]:
# Список классов COCO (для torchvision моделей)
COCO_CLASSES = [
    "person","bicycle","car","motorcycle","airplane","bus","train","truck","boat",
    "traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat",
    "dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack",
    "umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sports ball",
    "kite","baseball bat","baseball glove","skateboard","surfboard","tennis racket",
    "bottle","wine glass","cup","fork","knife","spoon","bowl","banana","apple",
    "sandwich","orange","broccoli","carrot","hot dog","pizza","donut","cake","chair",
    "couch","potted plant","bed","dining table","toilet","tv","laptop","mouse",
    "remote","keyboard","cell phone","microwave","oven","toaster","sink","refrigerator",
    "book","clock","vase","scissors","teddy bear","hair drier","toothbrush"
]

# Индекс класса "person"
PERSON_ID = COCO_CLASSES.index("person")   # будет равно 0

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [7]:
# Обнаруживаем объекты и сохраняем результирующие видео в отдельную папку
def detect_and_save(model_name, model, video_path, metrics_aggregator):
    cap = cv2.VideoCapture(str(video_path)) # открывает видеофайл для поочерёдного чтения кадров
    # сохраняем параметры видео (ширина, высота, частота кадров), чтобы выходное видео было с такими же
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) 

    out_path = RESULT_DIR / f"{model_name}_{video_path.name}" # сохраняем выходное видео
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # кодек для сохранения в mp4
    out = cv2.VideoWriter(str(out_path), fourcc, fps, (width, height)) # объект VideoWriter позволяет записать обработанные кадры в видео

    frame_count = 0
    people_counts, confidences, frame_times = [], [], []
    start_total = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        start_frame = time.time()

        people = []

        if model_name == "yolov8":
            results = model.predict(frame, conf=0.25, iou=0.45, device=DEVICE, verbose=False)[0]
            for box in results.boxes:
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                if model.names[cls] == "person":
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    people.append((x1, y1, x2, y2, conf))

        elif model_name == "yolov5":
            results = model(frame)
            for *xyxy, conf, cls in results.xyxy[0]:
                if model.names[int(cls)] == "person" and conf >= 0.25:
                    x1, y1, x2, y2 = map(int, xyxy)
                    people.append((x1, y1, x2, y2, float(conf)))

        else:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            tensor = torch.from_numpy(rgb).permute(2,0,1).float()/255.0
            tensor = tensor.unsqueeze(0).to(DEVICE)
            with torch.no_grad():
                outputs = model(tensor)[0]
            for box, score, label in zip(outputs["boxes"], outputs["scores"], outputs["labels"]):
                if label.item() == PERSON_ID and score > 0.25:
                    x1, y1, x2, y2 = map(int, box)
                    people.append((x1, y1, x2, y2, float(score)))

        # Рисуем рамки
        for (x1, y1, x2, y2, conf) in people:
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.putText(frame, f"person {conf:.2f}", (x1, max(0,y1-7)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

        # Метрики
        people_counts.append(len(people))
        confidences.extend([conf for *_, conf in people])
        frame_times.append(time.time() - start_frame)

        out.write(frame)

    cap.release()
    out.release()

    total_time = time.time() - start_total
    avg_frame_time = np.mean(frame_times) if frame_times else 0
    avg_fps = 1.0/avg_frame_time if avg_frame_time > 0 else 0
    avg_conf = np.mean(confidences) if confidences else 0
    avg_people = np.mean(people_counts) if people_counts else 0
    max_people = np.max(people_counts) if people_counts else 0
    total_detections = np.sum(people_counts)

    metrics = {
        "model": model_name,
        "video": video_path.name,
        "frames": frame_count,
        "avg_fps": round(avg_fps,2),
        "avg_frame_time_sec": round(avg_frame_time,4),
        "avg_confidence": round(avg_conf,2),
        "avg_people_per_frame": round(avg_people,2),
        "max_people_single_frame": int(max_people),
        "total_detections": int(total_detections),
        "total_time_sec": round(total_time,2),
    }
    metrics_aggregator.append(metrics)
    print(f"Сохранено в: {out_path}")

In [8]:
# Прогоняем все модели на трёх видосах

metrics_aggregator = []

for video_name in video_files:
    video_path = VIDEO_DIR / video_name
    for model_name, model in MODELS.items():
        detect_and_save(model_name, model, video_path, metrics_aggregator)

C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models

Сохранено в: results\yolov5_vid1.mp4
Сохранено в: results\yolov8_vid1.mp4
Сохранено в: results\ssd_vid1.mp4
Сохранено в: results\fasterrcnn_vid1.mp4
Сохранено в: results\retinanet_vid1.mp4
Сохранено в: results\maskrcnn_vid1.mp4


C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models

Сохранено в: results\yolov5_vid2.mp4
Сохранено в: results\yolov8_vid2.mp4
Сохранено в: results\ssd_vid2.mp4
Сохранено в: results\fasterrcnn_vid2.mp4
Сохранено в: results\retinanet_vid2.mp4
Сохранено в: results\maskrcnn_vid2.mp4


C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\gorod/.cache\torch\hub\ultralytics_yolov5_master\models

Сохранено в: results\yolov5_vid3.mp4
Сохранено в: results\yolov8_vid3.mp4
Сохранено в: results\ssd_vid3.mp4
Сохранено в: results\fasterrcnn_vid3.mp4
Сохранено в: results\retinanet_vid3.mp4
Сохранено в: results\maskrcnn_vid3.mp4


In [9]:
df = pd.DataFrame(metrics_aggregator)
print("\nИтоговая таблица сравнения:\n")
print(df)


Итоговая таблица сравнения:

         model     video  frames  avg_fps  avg_frame_time_sec  avg_confidence  \
0       yolov5  vid1.mp4    1500    14.22              0.0703            0.68   
1       yolov8  vid1.mp4    1500    20.38              0.0491            0.78   
2          ssd  vid1.mp4    1500     4.66              0.2144            0.00   
3   fasterrcnn  vid1.mp4    1500     1.24              0.8090            0.00   
4    retinanet  vid1.mp4    1500     1.22              0.8207            0.00   
5     maskrcnn  vid1.mp4    1500     0.98              1.0214            0.00   
6       yolov5  vid2.mp4    2653    25.33              0.0395            0.57   
7       yolov8  vid2.mp4    2653    27.54              0.0363            0.64   
8          ssd  vid2.mp4    2653     5.53              0.1809            0.00   
9   fasterrcnn  vid2.mp4    2653     1.34              0.7465            0.00   
10   retinanet  vid2.mp4    2653     1.34              0.7445            0.00  

Выводы:
1. YOLOv8, YOLOv5 - абсолютные победители. FPS - 14-27, также они дают достаточно уверенные детекции людей (avg_confidence от 0.54 до 0.78). Просмотрев результирующие видео с наложенными рамками, я убедилась, что количество детекций соответсвует реальному количеству людей в кадре. Помимо людей данные модели прекрасно справлялись с детекцией других объектов, таких как чашки, ложки, машины, кровати, часы и т.д. YOLOv8 чуть быстрее и точнее, особенно заметно на более динамчном видео.
2. Все модели из torchvision не справились с детекцией людей на видео. Можно было бы предположить, что проблема в слишком строгом пороге уверенности, который они не перешагивают, но avg_confidence по нулям... Скорее всего их надо дообучать, переобучать, но учитывая задачу - выявить наиболее пригодное для real-time детекции решение - даже если бы эти модели находили людей с их скоростью обработки видео они в любом случае непригодны.